In [ ]:
# Written By Suchith V

In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import requests
import re

In [ ]:
#set chromodriver.exe path
driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
#implicit wait
driver.implicitly_wait(0.5)
#maximize browser
driver.maximize_window()
url="https://www.swg-chemnitz.de/immobilien-vermarktungsart/miete/page/5/?post_type=immomakler_object&objekt-id&von-zimmer&bis-zimmer&von-qm&bis-qm&von-miete&bis-miete&collapse"
#launch URL
driver.get(url)
time.sleep(10)


In [2]:
requests.get(url)

<Response [200]>

In [8]:
content=driver.page_source.encode('utf-8').strip()
soup=BeautifulSoup(content,'html.parser')

In [123]:
urls_Raw=[]

for hrefs in soup.find_all('h3',attrs={'class':'property-title'}):
    urls_Raw.append(hrefs.a)
urls_Raw


[<a href="https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-16008-103-1-742/">Große Familienwohnung auf dem Kassberg!</a>,
 <a href="https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-20007-305-1-1215/">Mit EBK – ruhige 2-Zimmerwohnung in grüner Lage</a>,
 <a href="https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-12010-204-1-1604/">Traumgrundriss: 2-Raumwohnung nach Ihren Wünschen</a>,
 <a href="https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-11003-505-1-1416/">Tolle 2-Raumwohnung am Zeisigwald</a>,
 <a href="https://www.swg-chemnitz.de/mieten/wohnung-erdgeschosswohnung-in-chemnitz-mieten-12038-302-1-1602/">Schnell sein und wohl fühlen…hier wohnt es sich super!</a>,
 <a href="https://www.swg-chemnitz.de/mieten/wohnung-erdgeschosswohnung-in-chemnitz-mieten-22014-102-1-426/">Wohnen im Grünen! Idyllische 2- Raum-Wohnung mit Garten</a>,
 <a href="https://www.swg-chemnitz.de/mieten/wohn

In [124]:
#since urls list is not a list of strings, convert each element to strings
urls_To_Str=list(map(str,urls_Raw))

In [156]:
#As urls are quoted within "", we take advantage of this to filter url by splitting according to " usin split function
urls=[i.split('"', 2)[1] for i in urls_To_Str]
urls

['https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-16008-103-1-742/',
 'https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-20007-305-1-1215/',
 'https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-12010-204-1-1604/',
 'https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-11003-505-1-1416/',
 'https://www.swg-chemnitz.de/mieten/wohnung-erdgeschosswohnung-in-chemnitz-mieten-12038-302-1-1602/',
 'https://www.swg-chemnitz.de/mieten/wohnung-erdgeschosswohnung-in-chemnitz-mieten-22014-102-1-426/',
 'https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-25008-208-1-372/',
 'https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-22033-104-1-1395/',
 'https://www.swg-chemnitz.de/mieten/wohnung-erdgeschosswohnung-in-chemnitz-mieten-22033-101-1-1448/',
 'https://www.swg-chemnitz.de/mieten/wohnung-etagenwohnung-in-chemnitz-mieten-14059-110-1-1342/',
 'https:

In [147]:
# Now I want to go through this sites and extract information using beautifulsoup n times
len(urls)

58

In [279]:
def Execute_all(urls):
   
    #set chromodriver.exe path
    driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
    #implicit wait
    driver.implicitly_wait(0.5)
    #maximize browser
    driver.maximize_window()
    url=urls
    #launch URL
    driver.get(url)
    time.sleep(10)
    
    #Create Soup Object
    content1=driver.page_source.encode('utf-8').strip()
    soup1=BeautifulSoup(content1,'html.parser')
    
    #Parse The Data
    # Parse the Features
    Feature_Names=[]
    for hrefs_2 in soup1.find_all('div',attrs={'class':'dt col-sm-5'}):
        Feature_Names.append(hrefs_2.text)
    
    Features=[]
    for hrefs_21 in soup1.find_all('div',attrs={'class':'dd col-sm-7'}):
        Features.append(hrefs_21.text)
    # Zipping the two into dict    
    Combine_dict=dict(zip(Feature_Names,Features))    
    
    
    #Parse The Facilities
    fac_list_=[]
    for fac_list_raw in soup1.find_all('li',attrs={'class': 'list-group-item'}):
        fac_list_.append(fac_list_raw.text)
    
    fac_list_1=[]
    for fac_list_raw_1 in soup1.find_all('p'):
        fac_list_1.append(fac_list_raw_1.text)
    temp_fac=fac_list_1[1].split('-')
    
    #Unambiguous data is present. Extracting only required using regex
    #For regex, data should be in string format
    listToStr_Fac = ' '.join([str(i) for i in fac_list_])
    z=re.findall(r'✓\s\b[A-Z][a-z,ü]*',listToStr_Fac)
    facilities=[i.split(' ', 2)[1] for i in z]
    #Combining all facilities
    facilities=facilities+temp_fac
    
    return Combine_dict,facilities

In [280]:
#Executing For all urls
Nested_Features=[]
Nested_Facilities=[]
i=0
while i<len(url):
    Features_list,Facilities_list=Execute_all(urls[i])
    print(i)
    Nested_Features.append(Features_list)
    Nested_Facilities.append(Facilities_list)
    i=i+1
    

C:\Users\vsuch\AppData\Local\Temp\ipykernel_16524\2826490592.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")


0
1
2


KeyboardInterrupt: 

In [277]:
df_feat_swg=pd.DataFrame(Nested_Features)
ds_fac_swg=pd.Series(Nested_Facilities,name='Facilities')
df_swg=df_feat_swg.join(ds_fac_swg)
df_swg
df_swg.to_csv('Rentals_swg_58.csv')
# Code Terminates Here for SWG Site

In [278]:
df_swg


,Name,Adresse,Objekt ID,Objekttypen,Etage,Etagen im Haus,Wohnfläche ca.,Zimmer,Schlafzimmer,Badezimmer,...,Ausstattung,Verfügbar ab,Kaltmiete,Nebenkosten,Heizkosten,Warmmiete,Kaution,Nutzfläche ca.,Objekttyp,Facilities
0,Sächsische Wohnungsgenossenschaft Chemnitz eG,Reichsstraße 16a09112 Chemnitz (Kaßberg)Sachsen,16008.103 (1/742),"Etagenwohnung, Wohnung",1,4,"118,14 m²",4,3,2,...,gehoben,ab sofort,"750,00 EUR","142,00 EUR","85,00 EUR (in der Warmmiete enthalten)","977,00 EUR (Heizkosten enthalten)",NaN,NaN,NaN,"[Badewanne, Balkon, Dachboden, Dusche, Neubau,..."
1,Sächsische Wohnungsgenossenschaft Chemnitz eG,Terrassenstr. 3409131 Chemnitz (Hilbersdorf)Sa...,20007.305 (1/1215),"Etagenwohnung, Wohnung",NaN,2,"54,80 m²",2,1,1,...,NaN,01.02.2023,"307,00 EUR","67,00 EUR","84,00 EUR (in der Warmmiete enthalten)","458,00 EUR (Heizkosten enthalten)",NaN,NaN,NaN,"[Abstellraum, Altbau, Badewanne, Einbauküche, ..."
2,Sächsische Wohnungsgenossenschaft Chemnitz eG,Lortzingstraße 5109119 Chemnitz (Kapellenberg)...,12010.204 (1/1604),"Etagenwohnung, Wohnung",1,4,"60,70 m²",2,1,1,...,NaN,NaN,"371,00 EUR","73,00 EUR","43,00 EUR (in der Warmmiete enthalten)","487,00 EUR (Heizkosten enthalten)",NaN,NaN,NaN,"[Badewanne, Balkon, Dachboden, Dusche, Neubau,..."
3,Sächsische Wohnungsgenossenschaft Chemnitz eG,Terrassenstr. 4109131 Chemnitz (Hilbersdorf)Sa...,11003.505 (1/1416),"Etagenwohnung, Wohnung",2,3,"47,23 m²",2,1,1,...,NaN,ab sofort,"278,00 EUR","60,00 EUR","76,00 EUR (in der Warmmiete enthalten)","414,00 EUR (Heizkosten enthalten)",NaN,NaN,NaN,"[Balkon, Dusche, Neubau, Offene, Tageslichtbad..."
4,Sächsische Wohnungsgenossenschaft Chemnitz eG,Stollberger Straße 68b09119 Chemnitz (Kapellen...,12038_302 (1/1602),"Erdgeschosswohnung, Wohnung",EG,NaN,"58,18 m²",3,2,1,...,NaN,ab sofort,"332,00 EUR","122,00 EUR",NaN,"454,00 EUR","1.395,00 EUR",NaN,NaN,"[Balkon, Dusche, Neubau, Haben Sie Interesse a..."
5,Sächsische Wohnungsgenossenschaft Chemnitz eG,An der Kohlung 10b09114 Chemnitz (Glösa-Draisd...,22014.102 (1/426),"Erdgeschosswohnung, Wohnung",EG,2,"41,70 m²",2,1,1,...,NaN,ab sofort,"276,00 EUR","54,00 EUR","79,00 EUR (in der Warmmiete enthalten)","409,00 EUR (Heizkosten enthalten)",NaN,NaN,NaN,"[Altbau, Balkon, Dusche, Gartennutzung, Tagesl..."
6,Sächsische Wohnungsgenossenschaft Chemnitz eG,Paul-Arnold-Straße 1009130 Chemnitz (Sonnenber...,25008.208 (1/372),"Etagenwohnung, Wohnung",4,5,"53,65 m²",2,1,1,...,NaN,ab sofort,"274,00 EUR","68,00 EUR","49,00 EUR","391,00 EUR",NaN,NaN,NaN,"[Balkon, Dusche, , Balkon\n, Bad mit Dusche\..."
7,Sächsische Wohnungsgenossenschaft Chemnitz eG,Ziegeleistraße 109114 Chemnitz (Glösa-Draisdor...,22033.104 (1/1395),"Etagenwohnung, Wohnung",1,2,"45,87 m²",2,1,1,...,NaN,ab sofort,"271,00 EUR","59,00 EUR","87,00 EUR (in der Warmmiete enthalten)","417,00 EUR (Heizkosten enthalten)",NaN,NaN,NaN,"[Altbau, Dachboden, Dusche, Gartennutzung, Tag..."
8,Sächsische Wohnungsgenossenschaft Chemnitz eG,Ziegeleistraße 109114 Chemnitz (Glösa-Draisdor...,22033.101 (1/1448),"Erdgeschosswohnung, Wohnung",EG,3,"45,67 m²",2,1,1,...,NaN,ab sofort,"269,00 EUR","59,00 EUR","86,00 EUR (in der Warmmiete enthalten)","414,00 EUR (Heizkosten enthalten)",NaN,NaN,NaN,"[Altbau, Dachboden, Dusche, Gartennutzung, Tag..."
9,Sächsische Wohnungsgenossenschaft Chemnitz eG,Sonnenstraße 3509130 Chemnitz (Sonnenberg Süd)...,14059.110 (1/1342),"Etagenwohnung, Wohnung",5,5,"68,61 m²",3,2,1,...,NaN,ab sofort,"350,00 EUR","81,00 EUR","56,00 EUR (in der Warmmiete enthalten)","487,00 EUR (Heizkosten enthalten)",NaN,"68,61 m²",NaN,"[Abstellraum, Balkon, Dusche, Neubau, , Küche..."
